In [ ]:
#Sara Abu-Ghnaim
#A01 Final Project
#Experiment 625 Sandwich Stacker

#The object of this game is to stack a sandwich. The player controls 
#Experiment 625 (the animal moving on the bottom) and moves him horizontally with
#the mouse. Five objects will randomly fall from the sky, 3 of them edible
#(cheese, steak, and pickles) and 2 of them are not (shoes and fish skeletons). 
#For every edible item caught, a Sandwich Point will be added and a picture of 
#a smiley face will appear in the right panel. For every nonedible item caught,
#a point will be subtracted from the Sandwich points. The downward speed of the items
#will increase, making it harder and more confusing for the player. When the score
#reaches 5 points, the the speed will go back to what it was in the beginning of 
#the level and the score will start at 0. The right panel will also continue 
#to display the smiley faces.

import pygame, math, sys, random, os, time
from pygame.locals import *
import matplotlib.pyplot as plt

pygame.init()
screen = pygame.display.set_mode((800,600))
pygame.display.set_caption("Experiment 625 Sandwich Stacker")
pygame.mouse.set_visible(0) #1 = YES, 0 = NO

background = pygame.Surface(screen.get_size())
background = background.convert()
background.fill((250, 250, 250))

if pygame.font:
    font = pygame.font.Font(None, 36)
    text = font.render("Sandwich Stacker!!!", 1, (10, 10, 10))
    textpos = text.get_rect()
    textpos.centerx = background.get_rect().centerx
    background.blit(text, textpos)

screen.blit(background, (0, 0))
pygame.display.flip()

clock = pygame.time.Clock()

#Initialization and update Foodz
class Foodz (pygame.sprite.Sprite):
    def __init__(self, vel):
        self.on = True
        pygame.sprite.Sprite.__init__(self)
        x = random.random()
        if x <= 0.2:
            self.label = "cheese"
            self.image = pygame.image.load("cheese.png")
            self.image.set_colorkey((250,250,250))#making white transparent
            self.rect = self.image.get_rect()
            self.rect.topleft = int(random.random()*600), 0 #limits range in x up to 600
            self.speed = vel
        elif x > 0.2 and x <= 0.4:
            self.label = "pickle"
            self.image = pygame.image.load("pickle.jpg")
            self.image.set_colorkey((250,250,250))#making white transparent
            self.rect = self.image.get_rect()
            self.rect.topleft = int(random.random()*600), 0
            self.speed = vel
        elif x > 0.4 and x <= 0.6:
            self.label = "steak"
            self.image = pygame.image.load("steak.jpg")
            self.image.set_colorkey((250,250,250))#making white transparent
            self.rect = self.image.get_rect()
            self.rect.topleft = int(random.random()*600), 0
            self.speed = vel	
        elif x > 0.6 and x <= 0.8:
            self.label = "shoe"
            self.image = pygame.image.load("shoe.gif")
            self.image.set_colorkey((250,250,250))#making white transparent
            self.rect = self.image.get_rect()
            self.rect.topleft = int(random.random()*600), 0
            self.speed = vel
        elif x > 0.8:
            self.label = "fish"
            self.image = pygame.image.load("fish.jpg")
            self.image.set_colorkey((250,250,250))#making white transparent
            self.rect = self.image.get_rect()
            self.rect.topleft = int(random.random()*600), 0 
            self.speed = vel

    def update(self):
        newpos = self.rect.move((0,self.speed)) #Tuple with x_movement and y_movement
        self.rect = newpos
        if newpos[1] >= 600:
            self.on = False

#Initialization and Update Experiment 625, Stitch's cousin
class Ex625 (pygame.sprite.Sprite):

    def __init__(self):
        self.on = True
        pygame.sprite.Sprite.__init__(self)
        self.image = pygame.image.load("ex6251.jpg")
        self.image.set_colorkey((250,250,250))#making white transparent
        self.rect = self.image.get_rect()
        self.rect.bottomleft = 0, 0

    def update(self):
        newpos = pygame.mouse.get_pos()[0]
        self.rect = newpos - 0, 360

    def score(self,Foodz):
        cousin_x = self.rect[0]
        Foodz_x = Foodz.rect[0]

        if math.fabs(cousin_x-Foodz_x) <= 200: #within 200 pixels of each other
            return True
        else:
            return False

class Score (pygame.sprite.Sprite):

    def __init__(self):
        if pygame.font:
            self.on = True
            pygame.sprite.Sprite.__init__(self)
            self.font = pygame.font.Font(None, 34)
            self.image = self.font.render("Sandwich Points: {0}".format(0), 1, (10, 10, 10))
            self.rect = self.image.get_rect()
            self.rect.topright = 500, 300

    def update_score(self,score):
        self.image = self.font.render("Sandwich Points: {0}".format(score), 1, (10, 10, 10))
        self.rect = self.image.get_rect()
        self.rect.topright = 250, 0

class Strike (pygame.sprite.Sprite): #smileys appear from bottom right and stack on top

    def __init__(self,num):
        self.on = True
        pygame.sprite.Sprite.__init__(self)
        self.image = pygame.image.load("smile.jpg") #make into sandwich ingredients
        self.image.set_colorkey((250,250,250))#making white transparent
        self.rect = self.image.get_rect()
        ypos = 620-100*num
        self.rect.bottomright = 800, ypos

obj = []
foody = []
cousin = Ex625()
punti = Score()
obj.append(cousin)
obj.append(punti)
allobjects = pygame.sprite.RenderPlain(obj)

score = 0
strikes = 0
begin = time.clock()

#Main Loop
while 1:
    start_ticks = pygame.time.get_ticks() #starter tick
    seconds = (pygame.time.get_ticks()-start_ticks)/1000 #calculate how many seconds
    if seconds>20: # if more than 10 seconds close the game
        break
    clock.tick(100)
    allobjects.update()

    elapsed = time.clock() - begin

    #adding foodz if none present:
    if float(len(foody)) ==0:
        yayfood = Foodz(int(3*math.exp(0.2*elapsed)))
        obj.append(yayfood)
        foody.append(yayfood)
    #removing off object from obj
    for o in obj:
        if o.on == False:
            obj.remove(o)
            foody.remove(o)

    #checking if food item touches 625's picture
    for ff in foody:
        if ff.rect[1] >= 345: 
            if cousin.score(ff) == True: #checks if at same place
                if ff.label == "fish" or ff.label == "shoe":
                    score -= 1 #lose a point with each fish and shoe caught
                else:
                    score += 1 #catch food, score up 1, smile appears in right panel
                    strikes += 1
                    miss = Strike(strikes)
                    obj.append(miss)
            ff.on = False
        if score >= 5: #new round after with each 5 caught, objects fall slower again
                begin = time.clock()
                score = 0
    punti.update_score(score) #updating score in message 

    allobjects = pygame.sprite.RenderPlain(obj)

    screen.blit(background, (0, 0))
    allobjects.draw(screen)

    pygame.display.flip()

    for event in pygame.event.get():
        if event.type == K_ESCAPE:
            sys.exit(0)
            


pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:151: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:162: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:188: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
